In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers, callbacks



# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# read data

In [ ]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-feb-2022/train.csv', index_col=0)
test = pd.read_csv('/kaggle/input/tabular-playground-series-feb-2022/test.csv', index_col=0)
submission = pd.read_csv('/kaggle/input/tabular-playground-series-feb-2022/sample_submission.csv', index_col=0)

# check data

In [ ]:
cols_with_missing_train = [col for col in train.columns if train[col].isnull().any()]
cols_with_missing_test = [col for col in test.columns if test[col].isnull().any()]
print('train data missing vlue is : ',cols_with_missing_train)
print('test data missing vlue is : ',cols_with_missing_test)
print('train data shape is :',train.shape)
print('test data shape is :',test.shape)

# add some columns

In [ ]:
# Features with cardinality < 30
df_c=pd.concat([train,test])
c_list = train.nunique()[train.nunique() < 30][:-1]
print('categorical column is')
print(c_list)

In [ ]:
continus_train=train.drop(c_list.index,axis=1)
continus_train=continus_train.drop('target',axis=1)
train['mean']=continus_train.mean(axis=1)
train['min'] = continus_train.min(axis=1)
train['max'] = continus_train.max(axis=1)
train['std'] = continus_train.std(axis=1)

continus_test=test.drop(c_list.index,axis=1)
test['mean']=continus_test.mean(axis=1)
test['min'] = continus_test.min(axis=1)
test['max'] = continus_test.max(axis=1)
test['std'] = continus_test.std(axis=1)

In [ ]:
train.head()

# check duplicates

In [ ]:
print('duplicates count is :',train.duplicated().sum())

vc = train.value_counts()
print('unique count is :',len(vc))

In [ ]:
train = pd.DataFrame([list(tup) for tup in vc.index.values], columns=train.columns)
train['sample_weight'] = vc.values
weight = train['sample_weight']
train=train.drop('sample_weight',axis=1)
train.head()

In [ ]:
train.tail()

In [ ]:
weight.head()

# encode target

In [ ]:
a=(train.target).values
a=a.reshape(-1, 1)

# Apply one-hot encoder to each column with categorical data)
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(a))

# One-hot encoding removed index; put it back
OH_cols_train.index = train.index

# Remove categorical columns (will replace with one-hot encoding)
num_X_train = train.drop('target', axis=1)

# Add one-hot encoded columns to numerical features
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)

OH_X_train

In [ ]:
list=OH_encoder.categories_
list

In [ ]:
target=['Bacteroides_fragilis', 'Campylobacter_jejuni',
        'Enterococcus_hirae', 'Escherichia_coli', 'Escherichia_fergusonii',
        'Klebsiella_pneumoniae', 'Salmonella_enterica',
        'Staphylococcus_aureus', 'Streptococcus_pneumoniae',
        'Streptococcus_pyogenes']

# prepare training data

In [ ]:
X=OH_X_train.drop([0,1,2,3,4,5,6,7,8,9],axis=1)
y = OH_X_train[[0,1,2,3,4,5,6,7,8,9]]
#Separate data into training and validation sets
X_train, X_valid, y_train, y_valid, weight_train, weight_valid = train_test_split(X, y, weight, train_size=0.8, test_size=0.2,random_state=0)

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
weight_train.shape

# create model

In [ ]:

model = keras.Sequential([
    layers.BatchNormalization(),
    layers.Dense(80, activation='relu', input_shape=[290]),
    layers.Dropout(0.3),
    layers.BatchNormalization(),
    layers.Dense(28, activation='relu'),
    layers.Dropout(0.25),
    layers.BatchNormalization(),
    layers.Dense(10, activation='relu'),
    layers.Dense(10,activation='sigmoid'),
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
)


early_stopping = callbacks.EarlyStopping(
    min_delta=0.001, # minimium amount of change to count as an improvement
    patience=20, # how many epochs to wait before stopping
    restore_best_weights=True,
)


history = model.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid,weight_valid),
    batch_size=2000,
    epochs=200,
    callbacks=[early_stopping],
    sample_weight=weight_train,
)

# Show the learning curves
sns.set(rc = {'figure.figsize':(10,5)})
history_df = pd.DataFrame(history.history)
history_df.loc[:, ['loss', 'val_loss']].plot();

In [ ]:
a=model.predict(X_valid)
a = pd.DataFrame(data=a, columns=[0,1,2,3,4,5,6,7,8,9])
a=a.idxmax(axis=1)
a.index=y_valid.index
y_valid=y_valid.idxmax(axis=1)

print(accuracy_score(a,y_valid)*100)

# make prediction for submit

In [ ]:
prediction=model.predict(test)
prediction_df=pd.DataFrame(data=prediction, columns=[0,1,2,3,4,5,6,7,8,9])
prediction_df['target']=prediction_df.idxmax(axis=1)
prediction_df.index=test.index

In [ ]:
submission['row_id']=test.index
submission['target'] = prediction_df['target']
submission.head()

In [ ]:
submission.loc[submission['target']== 0, 'target'] = target[0]
submission.loc[submission['target']== 1, 'target'] = target[1]
submission.loc[submission['target']== 2, 'target'] = target[2]
submission.loc[submission['target']== 3, 'target'] = target[3]
submission.loc[submission['target']== 4, 'target'] = target[4]
submission.loc[submission['target']== 5, 'target'] = target[5]
submission.loc[submission['target']== 6, 'target'] = target[6]
submission.loc[submission['target']== 7, 'target'] = target[7]
submission.loc[submission['target']== 8, 'target'] = target[8]
submission.loc[submission['target']== 9, 'target'] = target[9]

In [ ]:
submission.head()

In [ ]:
submission.to_csv('submission.csv', index=False)